**1) 코사인 유사도(Cosine Similarity)**

1. 코사인 유사도(Cosine Similarity)

In [ ]:
import pandas as pd  # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
  '저는 사과 좋아요',
  '저는 바나나 좋아요',
  '저는 바나나 좋아요 저는 바나나 좋아요'
] 

vocab = list(set(w for doc in docs for w in doc.split()))
print('정렬 전 --------------------------------------')
print(vocab)
print()
vocab.sort()
print('정렬 후 --------------------------------------')
print(vocab)
print()

N = len(docs) # 총 문서의 수
print('총 문서 수 ------------------------------------')
print(N)
print()

# tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수.
def tf(t, d): 
  return d.count(t)


# df(t) : 특정 단어 t가 등장한 문서의 수.  
def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df + 1))

# idf(d, t) : df(t)에 반비례하는 수.
def tfidf(t, d):
  return tf(t,d)*idf(t)


result=[]
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns=vocab)
print('DTM ----------------------------------------')    
print(tf_)
print()
print(list(tf_.iloc[0]))
print(list(tf_.iloc[1]))
print(list(tf_.iloc[2]))

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

# 코사인 유사도를 구하는 함수
def cos_sim(A, B):
  return dot(A,B)/(norm(A) * norm(B))

# doc1=np.array([0,1,1,1])
# doc2=np.array([1,0,1,1])
# doc3=np.array([2,0,2,2])

doc1=list(tf_.iloc[0])
doc2=list(tf_.iloc[1])
doc3=list(tf_.iloc[2])


print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

2. 유사도를 이용한 추천 시스템 구현하기

In [ ]:
import pandas as pd

# 외부 데이터 읽기
import io
from google.colab import files
uploaded = files.upload()

data = pd.read_csv(io.StringIO(uploaded['movies_metadata.csv'].decode('utf-8')), low_memory=False)
print("data ------------------------------")
print(data.head(2))

In [ ]:
data = data.head(20000)

print(len(data))
print(data.columns)
print(data['overview'].isnull().sum())
print()

data['overview'] = data['overview'].fillna('') # overview에서 Null 값을 가진 경우에는 값 제거
print(data['overview'].isnull().sum())
print()

print('---------------------------------')
print(data[['title','overview']][:2])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview']) # overview에 대해서 tf-idf 수행

print(tfidf_matrix.shape)
print()


from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())
print()

idx = indices['Father of the Bride Part II']
print(idx)
print()

# 가장 overview가 유사한 10개의 영화
def get_recommendations(title, cosine_sim=cosine_sim):
  # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
  idx = indices[title]

  # 모든 영화에 대해서 해당 영화화의 유사도를 구합니다.
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 유사도에 따라 영화들을 정렬합니다.
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 가장 유사한 10개의 영화를 받아옵니다.
  sim_scores = sim_scores[1:11]

  # 가장 유사항 10개의 영화의 인덱스를 받아옵니다.
  movie_indices = [i[0] for i in sim_scores]

  # 가장 유사한 10개의 영화의 제목을 리턴합니다.
  return data['title'].iloc[movie_indices]

# print(data['title'].get(12481))

# movie_title = 'The Dark Knight Rises'
# movie_title = 'Toy Story'
movie_title = 'Man on the Moon'
print(movie_title)
print('overview 유사 -----------------------------') 
print(get_recommendations(movie_title))